# System of Linear Equations

## Question 1

In the classroom we have seen the LU decomposition with Doolitle's method,

Apply the algorithm by hand to find the solution to the problem
$Ax=b$ with:

$$A = \begin{bmatrix} 3 & -3 & 3 \\ -3 & 5 & 1 \\ 3 & 1 & 5 \end{bmatrix}$$

$$b^T = \begin{bmatrix} 9 & -7 & 12 \end{bmatrix}$$


Can you write down the algorithm in python ?
You can assume for now that the matrix is nice:
- all the pivot are not null
- there is no need to perform partial pivoting
- back substitution is smooth (none of the pivot are null)


In [1]:
import numpy as np

"""
LU decomposition corresponds to the Gauss elimination phase where we
store the multiplier in the lower triangular portion of A.
"""
def LUdecomp(a):
    n = len(a)
    for k in range(0, n-1):
        for i in range(k+1, n): 
            if a[i,k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
                a[i, k] = lam # Store the multiplier in the lower triangular
    return(a)

A = np.array([[3,-3,3], [-3,5,1], [3,1,5]], dtype=np.float64)
print("A = ")
print(A)
LU = LUdecomp(A)
print("LU = ")
print(LU)

"""
We must only compute LU once; it can then be used to compute Ax = LUx = b for any b.
Solve LUx = b by first solving Ly = b using forward substitution,
then Ux = y using backward substitution.
"""
def LUsolve(a, b):
    # a contains the matrix LU
    n = len(a)
    # Ly = b forward substitution (b is replaced by the solution of y)
    for k in range(1, n):
        b[k] = b[k] - np.dot(a[k, 0:k], b[0:k])
    # Ux = y backward substitution (b = y is replaced by the solution of x)
    b[n-1] = b[n-1] / a[n-1, n-1]
    for k in range(n-2, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k+1:n], b[k+1:n])) / a[k, k]
    return b

b = np.array([9,-7,12], dtype=np.float64)
print("b = ")
print(b)
print("x = ")
print(LUsolve(LU, b))

A = 
[[ 3. -3.  3.]
 [-3.  5.  1.]
 [ 3.  1.  5.]]
LU = 
[[ 3. -3.  3.]
 [-1.  2.  4.]
 [ 1.  2. -6.]]
b = 
[ 9. -7. 12.]
x = 
[3.5        0.66666667 0.16666667]


## Question 2

Do the same with a choleski decomposition using:

$$A = \begin{bmatrix} 2 & -1 & 0 \\ -1 & 2 & -1 \\ 0 & -1 & 2 \end{bmatrix}$$

$$b^T = \begin{bmatrix} 3 & -1 & 4 \end{bmatrix}$$

In [2]:
import numpy as np
import math

"""
Note that Choleski decomposition requires the matrix A to be
symmetric and positive definite.
"""
def choleskiDecomp(a):
    n = len(a)
    for k in range(n):
        try:
            a[k, k] = math.sqrt(a[k, k] - np.dot(a[k, 0:k], a[k, 0:k])) # Compute diagonal value
        except ValueError:
            raise Exception("Matrix is not positive definite")
        for i in range(k+1, n):
            a[i, k] = (a[i, k] - np.dot(a[i, 0:k], a[k, 0:k])) / a[k, k] # Compute off-diagonal values
    for k in range(1, n):
        a[0:k, k] = 0.0 # Erase upper triangle
    return a

A = np.array([[2,-1,0], [-1,2,-1], [0,-1,2]], dtype=np.float64)
print("A = ")
print(A)
L = choleskiDecomp(A)
print("L = ")
print(L)

"""
Solve Ax = L tr(L) x = b by first solving Ly = b using forward substitution,
then tr(L) x = y using backward substitution.
"""
def choleskiSolve(L, b):
    n = len(L)
    for k in range(n):
        b[k] = (b[k] - np.dot(L[k, 0:k], b[0:k])) / L[k, k]
    for k in range(n-1, -1, -1):
        b[k] = (b[k] - np.dot(L[k+1:n, k], b[k+1:n])) / L[k, k]
    return b

b = np.array([3,-1,4], dtype=np.float64)
print("b = ")
print(b)
print("x = ")
print(choleskiSolve(L, b))

A = 
[[ 2. -1.  0.]
 [-1.  2. -1.]
 [ 0. -1.  2.]]
L = 
[[ 1.41421356  0.          0.        ]
 [-0.70710678  1.22474487  0.        ]
 [ 0.         -0.81649658  1.15470054]]
b = 
[ 3. -1.  4.]
x = 
[2.75 2.5  3.25]


## Question 3

Improve your LU-decomposition algorithm (with Doolitle's), to include row pivoting

Hint:  you need to remember the row permutation that you applied in the decomposition phase and apply the same one on the 'b' vector in the solving phase.

Use your algorithm to solve the system defined by:

$$A = \begin{bmatrix} 0.6 & -0.4 & 1.0 \\ -0.3 & 0.2 & 0.5 \\ 0.6 & -1.0 & 0.5 \end{bmatrix}$$

$$b^T = \begin{bmatrix} 1.0 & 1.0 & 1.0 \end{bmatrix}$$


In [4]:
import numpy as np

"""
Perform pivoting and store the indices of the lines that where pivoted in rp
"""
def LUdecompRP(a):
    n = len(a)
    rp = []
    for k in range(0, n-1):
        pivot_idx = k + list(np.abs(a[k:n, k])).index(max(np.abs(a[k:n, k])))
        temp_a = a[k].copy()
        a[k] = a[pivot_idx]
        a[pivot_idx] = temp_a
        rp.append(pivot_idx)
        
        for i in range(k+1, n): 
            if a[i,k] != 0.0:
                lam = a[i, k] / a[k, k]
                a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
                a[i, k] = lam # Store the multiplier in the lower triangular
    return(a, rp)

A = np.array([[3,-3,3], [-3,5,1], [3,1,5]], dtype=np.float64)
print("A = ")
print(A)
LU, rp = LUdecompRP(A)
print("LU = ")
print(LU)
print("Pivoting order: " + str(rp))

"""
Iterate through rp to perform the pivoting of b before solving LUx = b
"""
def LUsolveRP(a, b, rp):
    n = len(a)
    for k in range(len(rp)):
        temp_b = b[k].copy()
        b[k] = b[rp[k]]
        b[rp[k]] = temp_b
        
    for k in range(1, n):
        b[k] = b[k] - np.dot(a[k, 0:k], b[0:k])
    b[n-1] = b[n-1] / a[n-1, n-1]
    for k in range(n-2, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k+1:n], b[k+1:n])) / a[k, k]
    return b

b = np.array([9,-7,12], dtype=np.float64)
print("b = ")
print(b)
print("x = ")
print(LUsolveRP(LU, b, rp))

A = 
[[ 3. -3.  3.]
 [-3.  5.  1.]
 [ 3.  1.  5.]]
LU = 
[[ 3.  -3.   3. ]
 [ 1.   4.   2. ]
 [-1.   0.5  3. ]]
Pivoting order: [0, 2]
b = 
[ 9. -7. 12.]
x = 
[3.5        0.66666667 0.16666667]


## Question 4

Compute the inverse of the following matrices:

$$A = \begin{bmatrix} 0.6 & -0.4 & 1.0 \\ -0.3 & 0.2 & 0.5 \\ 0.6 & -1.0 & 0.5 \end{bmatrix}$$

$$A = \begin{bmatrix} 1 & 2 & 4 \\ 1 & 3 & 9 \\ 1 & 4 & 16 \end{bmatrix}$$

$$A = \begin{bmatrix} 4 & -1 & 0 \\ -1 & 4 & -1 \\ 0 & -1 & 4 \end{bmatrix}$$

Which algorithm are you going to use and why ?

If we define the error on the inversion by:
    $error = max( AA^{-1} - I ) $

Are you getting smaller error when row pivoting is considered ?    

Solutions:

$A_1$ and $A_2$ are non-symmetric $\rightarrow$ Doolitle's. $A_3$ is symmetric $\rightarrow$ Choleski.


In [5]:
import numpy as np

A_1 = np.array([[0.6,-0.4,1.0], [-0.3,0.2,0.5], [0.6,-1.0,0.5]], dtype=np.float32)
A_2 = np.array([[1,2,4], [1,3,9], [1,4,16]], dtype=np.float32)
A_3 = np.array([[4,-1,0], [-1,4,-1], [0,-1,4]], dtype=np.float32)
b = np.array([[1,0,0],[0,1,0],[0,0,1]],dtype=np.float32)

print(np.max(A_1 @ LUsolve(LUdecomp(A_1.copy()),b.copy()) - b))
print(np.max(A_2 @ LUsolve(LUdecomp(A_2.copy()),b.copy()) - b))
print(np.max(A_3 @ LUsolve(LUdecomp(A_3.copy()),b.copy()) - b))

LU, rp =  LUdecompRP(A_1.copy())
Ainv = LUsolveRP(LU,b.copy(),rp)

print(np.max(A_1 @ Ainv - b))

LU, rp =  LUdecompRP(A_2.copy())
Ainv = LUsolveRP(LU,b.copy(),rp)

print(np.max(A_2 @ Ainv - b))

LU, rp =  LUdecompRP(A_3.copy())
Ainv = LUsolveRP(LU,b.copy(),rp)

print(np.max(A_3 @ Ainv - b))

nan
0.0
2.9802322e-08
5.9604645e-08
0.0
2.9802322e-08


/home/vhamer/.local/venvs/spyder/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in float_scalars
  if sys.path[0] == '':
/home/vhamer/.local/venvs/spyder/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


## Question 5

Can you write an algorithm to solve the following system using Gauss-Seidel algorithm without relaxation ?

$$A = \begin{bmatrix} 4 & -1 & 1 \\ -1 & 4 & -2 \\ 1 & -2 & 4 \end{bmatrix}$$

$$b^T = \begin{bmatrix} 12 & -1 & 5 \end{bmatrix}$$




In [6]:
import numpy as np

def gauss_seidel(A, b, epsilon=1e-10):
    n = A.shape[0]
    x = np.zeros(n)
    iter = 0
    diff = float("Inf")
    while(diff > epsilon):
        previous_x = x.copy()
        for i in range(0, n):
            x[i] = 1/A[i, i] * (b[i] - sum(A[i, 0:i] * x[0:i]) - sum(A[i, (i+1):n] * x[(i+1):n]))
                                                                            # Equivalent to use x or previous_x here
                                                                            # since x[i+1:n] has not been updated yet
        print(x)
        iter += 1
        diff = sum((x - previous_x)**2)
    return x
    
A = np.array([[4,-1,1], [-1,4,-2], [1,-2,4]], dtype=np.float32)
b = np.array([12,-1,5])

gauss_seidel(A,b)

[3.   0.5  0.75]
[2.9375    0.859375  0.9453125]
[2.97851562 0.96728516 0.98901367]
[2.99456787 0.9931488  0.99793243]
[2.99880409 0.99866724 0.9996326 ]
[2.99975866 0.99975596 0.99993832]
[2.99995441 0.99995776 0.99999028]
[2.99999187 0.99999311 0.99999859]
[2.99999863 0.99999895 0.99999982]


array([2.99999863, 0.99999895, 0.99999982])

## Question 6

Compute the inverse of the matrices defined in question Q4 using your Gauss-Seidel algorithm,
Compare the error you get with those obtained in Q4
Compare the CPU time difference to get the inverse using Gauss-Seidel and LU decomposition

Hint:  To get visible time difference repeat the inversion computation a large number of time (i.e. a thousand time)

In [16]:
import numpy as np
import time

def gauss_seidel(A, b, epsilon=1e-10):
    n = A.shape[0]
    x = np.zeros(n)
    iter = 0
    diff = float("Inf")
    while(diff > epsilon):
        previous_x = x.copy()
        for i in range(0,n):
            x[i] = 1/A[i, i] * (b[i] - sum(A[i,0:i] *x[0:i]) - sum(A[i,(i+1):n]*x[(i+1):n]))
        iter += 1
        diff = sum((x-previous_x)**2)
    return np.array([x])
    
A = np.array([[1,2,4], [1,3,9], [1,4,16]], dtype=np.float32)
b = np.identity(3)

start = time.time()

for i in range(0,100):
    x = np.concatenate((gauss_seidel(A_3,b[0,:]),gauss_seidel(A_3,b[1,:]),gauss_seidel(A_3,b[2,:])),axis=0)
    
print(x)
print(time.time()-start)

start = time.time()
for i in range(0,100):
    LU, rp =  LUdecompRP(A_1.copy())
    Ainv = LUsolveRP(LU,b.copy(),rp)
    
print(Ainv)
print(time.time()-start)

[[0.2678566  0.0714283  0.01785707]
 [0.0714283  0.28571415 0.07142854]
 [0.0178566  0.0714283  0.26785707]]
0.050612688064575195
[[ 1.66666658 -2.22222212 -1.11111104]
 [ 1.24999995 -0.8333333  -1.6666666 ]
 [ 0.5         1.          0.        ]]
0.0039560794830322266
